In [432]:
import math
import matplotlib_inline
import matplotlib as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error
from joblib import Parallel, delayed

from statsmodels.tsa.deterministic import DeterministicProcess

warnings.filterwarnings("ignore")

In [433]:
df_train = pd.read_csv("../dataset/train.csv", usecols=['store_nbr', 'family', 'date', 'sales'])
df_holiday = pd.read_csv("../dataset/holidays_events.csv")
df_oil = pd.read_csv("../dataset/oil.csv")
df_store = pd.read_csv("../dataset/stores.csv")

In [434]:
df_trn = df_train
df_trn["date"] = pd.to_datetime(df_train["date"])
df_trn = df_trn.set_index(['store_nbr', 'family', 'date']).sort_index()
display(df_trn.head())

sales
store_nbr family     date             
1         AUTOMOTIVE 2013-01-01    0.0
                     2013-01-02    2.0
                     2013-01-03    3.0
                     2013-01-04    3.0
                     2013-01-05    5.0

In [435]:
df_trnY = df_trn.unstack(['store_nbr', 'family'])
display(df_trnY.head())

sales                                                \
store_nbr          1                                                  
family     AUTOMOTIVE BABY CARE BEAUTY BEVERAGES BOOKS BREAD/BAKERY   
date                                                                  
2013-01-01        0.0       0.0    0.0       0.0   0.0        0.000   
2013-01-02        2.0       0.0    2.0    1091.0   0.0      470.652   
2013-01-03        3.0       0.0    0.0     919.0   0.0      310.655   
2013-01-04        3.0       0.0    3.0     953.0   0.0      198.366   
2013-01-05        5.0       0.0    3.0    1160.0   0.0      301.057   

                                                 ...                    \
store_nbr                                        ...        54           
family     CELEBRATION CLEANING  DAIRY     DELI  ... MAGAZINES   MEATS   
date                                             ...                     
2013-01-01         0.0      0.0    0.0    0.000  ...       0.0   0.000   
2013-01-02         0.0   1060.0  579.0  164.069  ...       0.0  73.771   
2013-01-03         0.0    836.0  453.0  151.582  ...       0.0  50.257   
2013-01-04         0.0    827.0  460.0  131.411  ...       0.0  40.223   
2013-01-05         0.0    811.0  464.0  118.613  ...       0.0  43.431   

                                                                       \
store_nbr                                                               
family     PERSONAL CARE PET SUPPLIES PLAYERS AND ELECTRONICS POULTRY   
date                                                                    
2013-01-01           0.0          0.0                     0.0   0.000   
2013-01-02         228.0          0.0                     0.0  15.514   
2013-01-03         156.0          0.0                     0.0   4.313   
2013-01-04         146.0          0.0                     0.0  26.743   
2013-01-05         205.0          0.0                     0.0  31.118   

                                                                      
store_nbr                                                             
family     PREPARED FOODS PRODUCE SCHOOL AND OFFICE SUPPLIES SEAFOOD  
date                                                                  
2013-01-01            0.0     0.0                        0.0     0.0  
2013-01-02           61.0     0.0                        0.0     3.0  
2013-01-03            1.0     0.0                        0.0     2.0  
2013-01-04           38.0     0.0                        0.0     2.0  
2013-01-05           32.0     0.0                        0.0     1.0  

[5 rows x 1782 columns]

In [436]:
df_oil["date"] = pd.to_datetime(df_oil["date"])
df_holiday["date"] = pd.to_datetime(df_holiday["date"])

df_holidayの重複を解除

In [437]:
df_holiday = df_holiday.loc[df_holiday["locale"] == "National", ["date", "type"]]
df_holiday = df_holiday.groupby("date").first()
df_holiday.reset_index(inplace=True)


df_trnXの作成

In [438]:
# dp = DeterministicProcess(index=df_trnY.index,
#                           constant=False,
#                           order=1,
#                           seasonal=True,
#                           period=5)
# df_trnX = dp.in_sample()                       

In [439]:
# df_trnX = df_trnX.merge(df_oil, on="date", how="left")
# df_trnX = df_trnX.merge(df_holiday[["date", "type"]], on="date", how="left")
# # Day of Week
# df_trnX["DoW"] = df_trnX["date"].dt.dayofweek
# df_trnX["dcoilwtico"].fillna(method="bfill", inplace=True)
# display(df_trnX.head())

In [440]:
df_trnX = pd.DataFrame()
df_trnX["date"] = df_trnY.index
df_trnX = df_trnX.merge(df_oil, on="date", how="left")
df_trnX = df_trnX.merge(df_holiday[["date", "type"]], on="date", how="left")
# Day of Week
df_trnX["DoW"] = df_trnX["date"].dt.dayofweek
df_trnX.set_index("date", inplace=True)
print(len(df_trnX))
df_trnX["dcoilwtico"].fillna(method="bfill", inplace=True)
display(df_trnX.head())

1684


,dcoilwtico,type,DoW
date,,,
2013-01-01,93.14,Holiday,1
2013-01-02,93.14,NaN,2
2013-01-03,92.97,NaN,3
2013-01-04,93.12,NaN,4
2013-01-05,93.20,Work Day,5


休みを表すisHoliday列を追加

In [441]:
df_trnX["isHoliday"] = 0
df_trnX.loc[df_trnX["DoW"] > 4, "isHoliday"] = 1
df_trnX.loc[df_trnX["type"] == "Work Day", "isHoliday"] = 0
df_trnX.loc[df_trnX["type"] == "Holiday", "isHoliday"] = 1
display(df_trnX.head())

,dcoilwtico,type,DoW,isHoliday
date,,,,
2013-01-01,93.14,Holiday,1,1
2013-01-02,93.14,NaN,2,0
2013-01-03,92.97,NaN,3,0
2013-01-04,93.12,NaN,4,0
2013-01-05,93.20,Work Day,5,0


カテゴリ変数 => ダミー変数

In [442]:
df_trnX = pd.get_dummies(df_trnX, columns=["DoW"], drop_first=True)
df_trnX = pd.get_dummies(df_trnX, columns=["type"], drop_first=False)
display(df_trnX.head())

,dcoilwtico,isHoliday,DoW_1,DoW_2,DoW_3,DoW_4,DoW_5,DoW_6,type_Additional,type_Bridge,type_Event,type_Holiday,type_Transfer,type_Work Day
date,,,,,,,,,,,,,,
2013-01-01,93.14,1,1,0,0,0,0,0,0,0,0,1,0,0
2013-01-02,93.14,0,0,1,0,0,0,0,0,0,0,0,0,0
2013-01-03,92.97,0,0,0,1,0,0,0,0,0,0,0,0,0
2013-01-04,93.12,0,0,0,0,1,0,0,0,0,0,0,0,0
2013-01-05,93.20,0,0,0,0,0,1,0,0,0,0,0,0,1


In [443]:
display(df_trnX.tail())
display(df_trnY.tail())

,dcoilwtico,isHoliday,DoW_1,DoW_2,DoW_3,DoW_4,DoW_5,DoW_6,type_Additional,type_Bridge,type_Event,type_Holiday,type_Transfer,type_Work Day
date,,,,,,,,,,,,,,
2017-08-11,48.81,0,0,0,0,1,0,0,0,0,0,0,1,0
2017-08-12,47.59,1,0,0,0,0,1,0,0,0,0,0,0,0
2017-08-13,47.59,1,0,0,0,0,0,1,0,0,0,0,0,0
2017-08-14,47.59,0,0,0,0,0,0,0,0,0,0,0,0,0
2017-08-15,47.57,0,1,0,0,0,0,0,0,0,0,0,0,0


sales                                                \
store_nbr          1                                                  
family     AUTOMOTIVE BABY CARE BEAUTY BEVERAGES BOOKS BREAD/BAKERY   
date                                                                  
2017-08-11        1.0       0.0    1.0    1006.0   0.0    145.60700   
2017-08-12        6.0       0.0    3.0    1659.0   0.0    243.22000   
2017-08-13        1.0       0.0    1.0     803.0   0.0    136.67900   
2017-08-14        1.0       0.0    6.0    2201.0   0.0    346.03800   
2017-08-15        4.0       0.0    4.0    1942.0   0.0    329.54102   

                                                 ...                       \
store_nbr                                        ...        54              
family     CELEBRATION CLEANING  DAIRY     DELI  ... MAGAZINES      MEATS   
date                                             ...                        
2017-08-11         4.0    341.0  343.0   64.302  ...       0.0  50.756000   
2017-08-12         3.0    351.0  526.0   99.488  ...       1.0  53.079002   
2017-08-13         1.0    169.0  266.0   47.770  ...       3.0  67.435000   
2017-08-14         4.0    571.0  699.0  154.578  ...       1.0  64.224000   
2017-08-15        21.0    703.0  602.0  116.402  ...       2.0  57.842000   

                                                                          \
store_nbr                                                                  
family     PERSONAL CARE PET SUPPLIES PLAYERS AND ELECTRONICS    POULTRY   
date                                                                       
2017-08-11         155.0          0.0                     0.0  80.759000   
2017-08-12         169.0          0.0                     4.0  91.671000   
2017-08-13         244.0          0.0                     2.0  79.062996   
2017-08-14         200.0          0.0                     1.0  56.155000   
2017-08-15         169.0          0.0                     2.0  59.619000   

                                                                       
store_nbr                                                              
family     PREPARED FOODS  PRODUCE SCHOOL AND OFFICE SUPPLIES SEAFOOD  
date                                                                   
2017-08-11           54.0  546.250                        0.0     0.0  
2017-08-12           81.0  696.920                        0.0     1.0  
2017-08-13           91.0  877.304                        0.0     2.0  
2017-08-14          147.0  585.615                        0.0     0.0  
2017-08-15           94.0  915.371                        0.0     3.0  

[5 rows x 1782 columns]

### モデル作成

In [444]:
model = Ridge(fit_intercept=True, solver="auto", alpha=0.5, normalize=True)
model.fit(df_trnX, df_trnY)
y_pred = pd.DataFrame(model.predict(df_trnX), index=df_trnX.index, columns=df_trnY.columns)

In [445]:
y_pred   = y_pred.stack(['store_nbr', 'family']).reset_index()
y_target = y.stack(['store_nbr', 'family']).reset_index().copy()

y_target['sales_pred'] = y_pred['sales'].clip(0.)

y_target.groupby('family').apply(lambda x: mean_squared_log_error(x['sales'], x['sales_pred']))

family
AUTOMOTIVE                     0.401609
BABY CARE                      0.052717
BEAUTY                         0.335486
BEVERAGES                      3.141745
BOOKS                          0.039998
BREAD/BAKERY                   1.684145
CELEBRATION                    1.583205
CLEANING                       2.296501
DAIRY                          2.005933
DELI                           1.351245
EGGS                           1.082541
FROZEN FOODS                   1.157069
GROCERY I                      3.472921
GROCERY II                     0.583881
HARDWARE                       0.300294
HOME AND KITCHEN I             2.222023
HOME AND KITCHEN II            1.635891
HOME APPLIANCES                0.159796
HOME CARE                      8.603127
LADIESWEAR                     1.337895
LAWN AND GARDEN                0.580733
LINGERIE                       0.571808
LIQUOR,WINE,BEER               2.373514
MAGAZINES                      0.590894
MEATS                          1.

In [446]:
class SalesRegressor():
    
    def __init__(self, n_jobs=-1, verbose=0):
        
        self.n_jobs = n_jobs
        self.verbose = verbose
        
        self.estimators_ = None
        
    def _estimator_(self, X, y):
    
        warnings.simplefilter(action='ignore', category=FutureWarning)
        
        if y.name[2] == 'SCHOOL AND OFFICE SUPPLIES':
            
            model = RandomForestRegressor(n_estimators = 300, n_jobs=-1, random_state=1)
            
        else:
            
            model = Ridge(fit_intercept=True, solver='auto', alpha=0.5, normalize=True)
            
        model.fit(X, y)

        return model

    def fit(self, X, y):

        self.estimators_ = Parallel(n_jobs=self.n_jobs, 
                              verbose=self.verbose,
                              )(delayed(self._estimator_)(X, y.iloc[:, i]) for i in range(y.shape[1]))
        
        return
    
    def predict(self, X):
        
        y_pred = Parallel(n_jobs=self.n_jobs, 
                          verbose=self.verbose)(delayed(e.predict)(X) for e in self.estimators_)
        
        return np.stack(y_pred, axis=1)

In [447]:
model = SalesRegressor(n_jobs=-1, verbose=0)
model.fit(df_trnX, df_trnY)
y_pred = pd.DataFrame(model.predict(df_trnX), index=df_trnX.index, columns=df_trnY.columns)

In [448]:
y_pred   = y_pred.stack(['store_nbr', 'family']).reset_index()
y_target = y.stack(['store_nbr', 'family']).reset_index().copy()
y_target['sales_pred'] = y_pred['sales'].clip(0.)
y_target.groupby('family').apply(lambda x: mean_squared_log_error(x['sales'], x['sales_pred']))

family
AUTOMOTIVE                     0.401609
BABY CARE                      0.052717
BEAUTY                         0.335486
BEVERAGES                      3.141745
BOOKS                          0.039998
BREAD/BAKERY                   1.684145
CELEBRATION                    1.583205
CLEANING                       2.296501
DAIRY                          2.005933
DELI                           1.351245
EGGS                           1.082541
FROZEN FOODS                   1.157069
GROCERY I                      3.472921
GROCERY II                     0.583881
HARDWARE                       0.300294
HOME AND KITCHEN I             2.222023
HOME AND KITCHEN II            1.635891
HOME APPLIANCES                0.159796
HOME CARE                      8.603127
LADIESWEAR                     1.337895
LAWN AND GARDEN                0.580733
LINGERIE                       0.571808
LIQUOR,WINE,BEER               2.373514
MAGAZINES                      0.590894
MEATS                          1.

### テストデータの作成

In [449]:
df_test = pd.read_csv("../dataset/test.csv", usecols=['store_nbr', 'family', 'date'])
df_test["date"] = pd.to_datetime(df_test["date"])
df_test = df_test.set_index(['store_nbr', 'family', 'date']).sort_index()
display(df_test.head())

Empty DataFrame
Columns: []
Index: [(1, AUTOMOTIVE, 2017-08-16 00:00:00), (1, AUTOMOTIVE, 2017-08-17 00:00:00), (1, AUTOMOTIVE, 2017-08-18 00:00:00), (1, AUTOMOTIVE, 2017-08-19 00:00:00), (1, AUTOMOTIVE, 2017-08-20 00:00:00)]

In [450]:
start = "2017-08-16"
end = "2017-08-31"

In [456]:
df_testX = pd.DataFrame()
df_testX["date"] = df_test.index.get_level_values("date").unique()
print(df_testX.info())
df_testX = df_testX.merge(df_oil, on="date", how="left")

# Day of Week
df_testX["DoW"] = df_testX["date"].dt.dayofweek
df_testX.set_index("date", inplace=True)
df_testX["dcoilwtico"].fillna(method="bfill", inplace=True)

df_testX["isHoliday"] = 0
df_testX.loc[df_testX["DoW"] > 4, "isHoliday"] = 1

df_testX = pd.get_dummies(df_testX, columns=["DoW"], drop_first=True)

# df_testの期間にholidayがないため0埋め
df_testX[["type_Additional","type_Bridge","type_Event", "type_Holiday", "type_Transfer", "type_Work Day"]] = 0



display(df_testX.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    16 non-null     datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 256.0 bytes
None


,dcoilwtico,isHoliday,DoW_1,DoW_2,DoW_3,DoW_4,DoW_5,DoW_6,type_Additional,type_Bridge,type_Event,type_Holiday,type_Transfer,type_Work Day
date,,,,,,,,,,,,,,
2017-08-16,46.80,0,0,1,0,0,0,0,0,0,0,0,0,0
2017-08-17,47.07,0,0,0,1,0,0,0,0,0,0,0,0,0
2017-08-18,48.59,0,0,0,0,1,0,0,0,0,0,0,0,0
2017-08-19,47.39,1,0,0,0,0,1,0,0,0,0,0,0,0
2017-08-20,47.39,1,0,0,0,0,0,1,0,0,0,0,0,0


In [457]:
sales_pred = pd.DataFrame(model.predict(df_testX), index=df_testX.index, columns=df_trnY.columns)
display(sales_pred.head())
sales_pred = sales_pred.stack(['store_nbr', 'family'])
display(sales_pred.head())
sales_pred[sales_pred < 0] = 0.
display(sales_pred.head())

sales                                                          \
store_nbr          1                                                            
family     AUTOMOTIVE BABY CARE    BEAUTY    BEVERAGES     BOOKS BREAD/BAKERY   
date                                                                            
2017-08-16   3.885506       0.0  2.797012  2014.000448  0.201532   425.595949   
2017-08-17   3.555434       0.0  2.752281  1821.488257  0.157081   385.330130   
2017-08-18   4.082873       0.0  2.636323  1931.755182  0.184223   392.421233   
2017-08-19   3.854221       0.0  2.781521  1917.458356  0.162267   361.406123   
2017-08-20   2.239027       0.0  1.764019  1199.077143  0.123537   219.356780   

                                                            ...            \
store_nbr                                                   ...        54   
family     CELEBRATION    CLEANING       DAIRY        DELI  ... MAGAZINES   
date                                                        ...             
2017-08-16   13.746935  814.210955  796.678397  144.680022  ...  0.535754   
2017-08-17   13.978759  702.403479  678.258467  129.180802  ...  0.523369   
2017-08-18   15.426390  739.191059  723.258572  153.319414  ...  0.597574   
2017-08-19   11.032960  602.498502  711.587489  133.258853  ...  0.812978   
2017-08-20    6.843005  356.267911  444.570236   81.526185  ...  0.740324   

                                                                          \
store_nbr                                                                  
family          MEATS PERSONAL CARE PET SUPPLIES PLAYERS AND ELECTRONICS   
date                                                                       
2017-08-16  45.168883    161.567879     0.133444                2.179304   
2017-08-17  54.311571    153.596668     0.112943                2.147646   
2017-08-18  49.104270    168.020984     0.103902                2.389016   
2017-08-19  58.248979    226.453144     0.155839                2.863001   
2017-08-20  67.394281    287.359840     0.159298                3.089161   

                                                                             \
store_nbr                                                                     
family        POULTRY PREPARED FOODS     PRODUCE SCHOOL AND OFFICE SUPPLIES   
date                                                                          
2017-08-16  46.937610      61.952451  404.036294                   0.000000   
2017-08-17  48.157657      63.789406  396.716711                   0.093333   
2017-08-18  56.830795      62.123239  411.806465                   0.486667   
2017-08-19  59.046514      63.936294  488.792298                   0.000000   
2017-08-20  67.185841      82.305064  532.623087                   0.005000   

                      
store_nbr             
family       SEAFOOD  
date                  
2017-08-16  1.760733  
2017-08-17  1.900288  
2017-08-18  1.337137  
2017-08-19  1.737896  
2017-08-20  2.172052  

[5 rows x 1782 columns]

sales
date       store_nbr family                 
2017-08-16 1         AUTOMOTIVE     3.885506
                     BABY CARE      0.000000
                     BEAUTY         2.797012
                     BEVERAGES   2014.000448
                     BOOKS          0.201532

sales
date       store_nbr family                 
2017-08-16 1         AUTOMOTIVE     3.885506
                     BABY CARE      0.000000
                     BEAUTY         2.797012
                     BEVERAGES   2014.000448
                     BOOKS          0.201532

In [458]:
df_output = pd.read_csv('../dataset/sample_submission.csv', index_col='id')
df_output["sales"] = sales_pred.values
df_output.to_csv('./output.csv')